# Automated ComfyUI on Brev

This notebook is modified from ...

In [ ]:
# Environment Setup

from pathlib import Path

OPTIONS = {}

WORKSPACE = 'ComfyUI'

#@markdown **Custom Nodes Repositories**
NODES_REPOS = [
    'https://github.com/ltdrdata/ComfyUI-Manager.git',
    'https://github.com/ltdrdata/ComfyUI-Impact-Pack.git',
    'https://github.com/ltdrdata/ComfyUI-Impact-Subpack.git',
    'https://github.com/Suzie1/ComfyUI_Comfyroll_CustomNodes.git'
]

CIVITAI_MODELS = []
HUGGINGFACE_MODELS = []

OPTIONS['NODES_REPOS'] = NODES_REPOS
OPTIONS['CIVITAI_MODELS'] = CIVITAI_MODELS
OPTIONS['HUGGINGFACE_MODELS'] = HUGGINGFACE_MODELS

In [ ]:
# Helper to download repos

import os
import re
import subprocess
import time
from typing import List, Optional

def clone_repo(repo_url: str, target_dir: str) -> None:
    """Clone a Git repository to the target directory."""
    # Extract repo name from URL (works with both HTTPS and SSH URLs)
    repo_name = None
    
    # Try to match patterns for extracting repository name
    https_pattern = r'.*/.*/(.*)\.git$'
    simple_pattern = r'.*/(.*)$'
    
    match = re.match(https_pattern, repo_url)
    if match:
        repo_name = match.group(1)
    else:
        match = re.match(simple_pattern, repo_url)
        if match:
            repo_name = match.group(1)
    
    # If we can't extract a name, use a timestamp
    if not repo_name:
        repo_name = f"custom_repo_{int(time.time())}"
    
    dest_path = os.path.join(target_dir, repo_name)
    print(f"Cloning repository: {repo_url} to {dest_path}")
    
    # Clone the repository using subprocess
    try:
        result = subprocess.run(
            ["git", "clone", repo_url, dest_path],
            check=True,
            capture_output=True,
            text=True
        )
        print(f"Successfully cloned {repo_url}")
    except subprocess.CalledProcessError as e:
        print(f"Failed to clone {repo_url}")
        print(f"Error: {e.stderr}")

In [ ]:
# Clone ComfyUI

target_dir = ""
repo = "https://github.com/comfyanonymous/ComfyUI"

clone_repo(repo, target_dir)

In [ ]:
# Install Requirements
import os

%pip install -r ComfyUI/requirements.txt

Now move on to the custom nodes for this install

In [ ]:

# Get the target directory from environment variable or use a default
target_dir = "ComfyUI/custom_nodes"

# Check if repositories are specified in OPTIONS
if 'NODES_REPOS' in OPTIONS and OPTIONS['NODES_REPOS']:
    repos = OPTIONS['NODES_REPOS']  # This is already an array of GitHub repos
    print(f"Cloning {len(repos)} repositories from OPTIONS['NODES_REPOS']")
    
    for repo in repos:
        if repo:  # Make sure the repo URL is not empty
            clone_repo(repo, target_dir)
else:
    print("No custom nodes specified in OPTIONS['NODES_REPOS']")

In [ ]:
import os
import subprocess
import glob

# Get all directories within custom_nodes/
custom_node_dirs = glob.glob('custom_nodes/*/')

for dir_path in custom_node_dirs:
    req_file = os.path.join(dir_path, 'requirements.txt')
    
    # Check if requirements.txt exists in the directory
    if os.path.isfile(req_file):
        print(f"Installing requirements in {dir_path}")
        # Run pip install for the requirements file
        subprocess.check_call(['pip', 'install', '-r', req_file])

In [ ]:
import os
!pip install huggingface_hub

In [ ]:
!wget -c https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt -P ./models/checkpoints/

In [ ]:
!wget -c https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors -P ./models/vae/

In [ ]:
import subprocess
import threading
import time
import socket

def check_server(port):
    while True:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
            result = sock.connect_ex(('127.0.0.1', port))
            if result == 0:
                print("\nServer is up and running at http://129.80.69.99:{}".format(port))
                break
            time.sleep(0.5)

threading.Thread(target=check_server, daemon=True, args=(8188,)).start()

In [ ]:
!python main.py --listen 0.0.0.0 --port 8188 --dont-print-server --disable-xformers

In [ ]:
def check_server(port):
    while True:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
            result = sock.connect_ex(('127.0.0.1', port))
            if result == 0:
                print("\nServer is up and running at http://129.80.69.99:{}".format(port))
                break
            time.sleep(0.5)

threading.Thread(target=check_server, daemon=True, args=(8188,)).start()